<a href="https://colab.research.google.com/github/jwaes/ZX2D10GE01R-V4848/blob/master/netcalc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime

# Define your filename
filename = "Verbruikshistoriek_elektriciteit_541448820047183849_20230505_20250204_kwartiertotalen.csv"

# Split the filename into parts
parts = filename.split('_')

# Extract the variables from the parts
ean_number    = parts[2]  # "541448820047183849"
start_date_str = parts[3]  # "20230505"
end_date_str   = parts[4]  # "20250204"

# Convert the date strings into datetime objects
start_date = datetime.strptime(start_date_str, "%Y%m%d")
end_date   = datetime.strptime(end_date_str, "%Y%m%d")

print("EAN Number:", ean_number)
print("Start Date:", start_date)
print("End Date:", end_date)

EAN Number: 541448820047183849
Start Date: 2023-05-05 00:00:00
End Date: 2025-02-04 00:00:00


In [2]:
import pandas as pd

# Read the CSV file with the proper delimiter and decimal settings
df = pd.read_csv(
    filename,
    delimiter=';',
    decimal=','
)

# Preview the first few rows
print(df.head())

# --- Cleaning the Data ---

# Clean the EAN-code column by removing the extra characters (=" and ")
df['EAN-code'] = df['EAN-code'].astype(str).str.replace('="', '', regex=False).str.replace('"', '', regex=False)

# Combine the date and time columns for the "start" timestamps into a single datetime column
df['Start_datetime'] = pd.to_datetime(
    df['Van (datum)'] + ' ' + df['Van (tijdstip)'],
    format='%d-%m-%Y %H:%M:%S'
)

# Similarly, combine the "end" date and time columns into a single datetime column
df['End_datetime'] = pd.to_datetime(
    df['Tot (datum)'] + ' ' + df['Tot (tijdstip)'],
    format='%d-%m-%Y %H:%M:%S'
)

# Convert the "Volume" column to a numeric type; any non-numeric entries will be set as NaN
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

# Check the data types to ensure everything is ready for calculations
print(df.info())


  Van (datum) Van (tijdstip) Tot (datum) Tot (tijdstip)  \
0  05-05-2023       00:00:00  05-05-2023       00:15:00   
1  05-05-2023       00:00:00  05-05-2023       00:15:00   
2  05-05-2023       00:15:00  05-05-2023       00:30:00   
3  05-05-2023       00:15:00  05-05-2023       00:30:00   
4  05-05-2023       00:30:00  05-05-2023       00:45:00   

                EAN-code           Meter       Metertype        Register  \
0  ="541448820047183849"  1SAG3105019704  Digitale meter    Afname Nacht   
1  ="541448820047183849"  1SAG3105019704  Digitale meter  Injectie Nacht   
2  ="541448820047183849"  1SAG3105019704  Digitale meter    Afname Nacht   
3  ="541448820047183849"  1SAG3105019704  Digitale meter  Injectie Nacht   
4  ="541448820047183849"  1SAG3105019704  Digitale meter    Afname Nacht   

   Volume Eenheid Validatiestatus   Omschrijving  
0     NaN     kWh   Geen verbruik  Hoofdverblijf  
1     NaN     kWh   Geen verbruik  Hoofdverblijf  
2   0.045     kWh      Uitgelezen  

In [3]:
# prompt: describe the ods134.parquet file content

import pandas as pd

# Assuming 'ods134.parquet' is in the current working directory
try:
    df_parquet = pd.read_parquet('ods134.parquet')
    print(df_parquet.info())  # Print info about columns, data types, non-null counts, etc.
    print(df_parquet.head())   # Show the first few rows of the DataFrame
    print(df_parquet.describe()) # Descriptive statistics for numerical columns
except FileNotFoundError:
    print("Error: 'ods134.parquet' not found. Please make sure the file exists in the current directory or provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24700 entries, 0 to 24699
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype                          
---  ------                    --------------  -----                          
 0   datetime                  24700 non-null  datetime64[ms, Europe/Brussels]
 1   resolutioncode            24700 non-null  object                         
 2   qualitystatus             24700 non-null  object                         
 3   ace                       24700 non-null  float64                        
 4   systemimbalance           24700 non-null  float64                        
 5   alpha                     24700 non-null  float64                        
 6   alpha_prime               20860 non-null  float64                        
 7   marginalincrementalprice  24700 non-null  float64                        
 8   marginaldecrementalprice  24700 non-null  float64                        
 9   imbalanceprice   

In [10]:
# prompt: describe the ods047.parquet file content

import pandas as pd

# Assuming 'ods047.parquet' is in the current working directory
try:
    df_parquet = pd.read_parquet('ods047.parquet')
    print(df_parquet.info())  # Print info about columns, data types, non-null counts, etc.
    print(df_parquet.head())   # Show the first few rows of the DataFrame
    print(df_parquet.describe()) # Descriptive statistics for numerical columns
except FileNotFoundError:
    print("Error: 'ods047.parquet' not found. Please make sure the file exists in the current directory or provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329180 entries, 0 to 329179
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype                          
---  ------                    --------------   -----                          
 0   datetime                  329180 non-null  datetime64[ms, Europe/Brussels]
 1   qualitystatus             329180 non-null  object                         
 2   resolutioncode            329180 non-null  object                         
 3   netregulationvolume       329180 non-null  float64                        
 4   systemimbalance           329180 non-null  float64                        
 5   alpha                     329180 non-null  float64                        
 6   marginalincrementalprice  329180 non-null  float64                        
 7   marginaldecrementalprice  329180 non-null  float64                        
 8   strategicreserveprice     716 non-null     float64                        
 9   posi

In [7]:
# prompt: create a joined dataset consisting of the common columns of both ods134.parquet and ods047.parquet files

import pandas as pd

try:
    df_ods134 = pd.read_parquet('ods134.parquet')
    df_ods047 = pd.read_parquet('ods047.parquet')

    # Find common columns
    common_cols = list(set(df_ods134.columns) & set(df_ods047.columns))

    # Create joined dataset
    if common_cols:
        joined_df = pd.merge(df_ods134[common_cols], df_ods047[common_cols], how='inner', on=common_cols)
        print(joined_df.info())  # Print info about columns, data types, non-null counts, etc.
        print(joined_df.head())   # Show the first few rows of the DataFrame
        print(joined_df.describe()) # Descriptive statistics for numerical columns
    else:
        print("No common columns found between the two datasets.")

except FileNotFoundError:
    print("Error: One or both parquet files not found.  Please ensure 'ods134.parquet' and 'ods047.parquet' are in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype                          
---  ------                    --------------  -----                          
 0   alpha                     0 non-null      float64                        
 1   qualitystatus             0 non-null      object                         
 2   resolutioncode            0 non-null      object                         
 3   marginaldecrementalprice  0 non-null      float64                        
 4   systemimbalance           0 non-null      float64                        
 5   marginalincrementalprice  0 non-null      float64                        
 6   datetime                  0 non-null      datetime64[ms, Europe/Brussels]
dtypes: datetime64[ms, Europe/Brussels](1), float64(4), object(2)
memory usage: 132.0+ bytes
None
Empty DataFrame
Columns: [alpha, qualitystatus, resolutioncode, marginaldecrementalprice, systemimb

In [12]:
import pandas as pd

try:
    df_ods134 = pd.read_parquet('ods134.parquet')
    df_ods047 = pd.read_parquet('ods047.parquet')

    # Find common columns between the two DataFrames
    common_cols = list(set(df_ods134.columns) & set(df_ods047.columns))

    if common_cols:
        # Concatenate the two DataFrames, keeping only the common columns
        joined_df = pd.concat([df_ods134[common_cols], df_ods047[common_cols]], ignore_index=True)
        print(joined_df.info())   # Print info about columns, data types, non-null counts, etc.
        print(joined_df.head())   # Show the first few rows of the DataFrame
        print(joined_df.describe())  # Descriptive statistics for numerical columns
    else:
        print("No common columns found between the two datasets.")

except FileNotFoundError:
    print("Error: One or both parquet files not found. Please ensure 'ods134.parquet' and 'ods047.parquet' are in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353880 entries, 0 to 353879
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype                          
---  ------                    --------------   -----                          
 0   alpha                     353880 non-null  float64                        
 1   qualitystatus             353880 non-null  object                         
 2   resolutioncode            353880 non-null  object                         
 3   marginaldecrementalprice  353880 non-null  float64                        
 4   systemimbalance           353880 non-null  float64                        
 5   marginalincrementalprice  353880 non-null  float64                        
 6   datetime                  353880 non-null  datetime64[ms, Europe/Brussels]
dtypes: datetime64[ms, Europe/Brussels](1), float64(4), object(2)
memory usage: 18.9+ MB
None
   alpha qualitystatus resolutioncode  marginaldecrementalprice  \
0  0.000 